시험용 데이터를 나누는 이유는 과적합 여부 등 모델의 성능을 평가하기 위함  
편향현상과 같은 이상 결과를 피하기 위해 훈련용 데이터셋을 다시 훈련용과 검증용으로 나눔

# 타이타닉 데이터

In [2]:
import pandas as pd

titanic_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master' +\
                                    '/dataset/titanic.xls'

titanic = pd.read_excel(titanic_url)
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [5]:
# 라벨인코더로 성별 컬럼 만들기

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(titanic['sex'])
titanic['gender'] = le.transform(titanic['sex'])
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,gender
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",0
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",1
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",0
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON",1
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",0


In [6]:
# 숫자가 아닌 데이터 걸러내기

titanic = titanic[titanic['age'].notnull()]
titanic = titanic[titanic['fare'].notnull()]
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1045 entries, 0 to 1308
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1045 non-null   int64  
 1   survived   1045 non-null   int64  
 2   name       1045 non-null   object 
 3   sex        1045 non-null   object 
 4   age        1045 non-null   float64
 5   sibsp      1045 non-null   int64  
 6   parch      1045 non-null   int64  
 7   ticket     1045 non-null   object 
 8   fare       1045 non-null   float64
 9   cabin      272 non-null    object 
 10  embarked   1043 non-null   object 
 11  boat       417 non-null    object 
 12  body       119 non-null    float64
 13  home.dest  685 non-null    object 
 14  gender     1045 non-null   int64  
dtypes: float64(3), int64(5), object(7)
memory usage: 130.6+ KB


In [7]:
# 전체 라벨 데이터와 특성 데이터 추출

y = titanic['survived'].values
X = titanic[['pclass', 'age', 'sibsp', 'parch', 'fare', 'gender']].values

In [9]:
# StandScaler 적용

from sklearn.preprocessing import StandardScaler

SS = StandardScaler()
SS.fit(X)

X_ss = SS.transform(X)

In [11]:
# LogisticRegression과 KFold 불러오기

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

log_reg_kf = LogisticRegression(random_state=13, 
                               solver='liblinear')
kfold = KFold(n_splits=5)
cv_accuracy = []

In [12]:
# 교차 검증 수행

from sklearn.metrics import accuracy_score

for train_idx, test_idx in kfold.split(X_ss):
    X_train, X_test = X_ss[train_idx], X_ss[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    log_reg_kf.fit(X_train, y_train)
    
    pred = log_reg_kf.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    
    cv_accuracy.append(accuracy)

In [13]:
cv_accuracy

[0.7655502392344498,
 0.8660287081339713,
 0.8229665071770335,
 0.7464114832535885,
 0.7081339712918661]

In [14]:
# 교차 검증의 평균값
import numpy as np

np.mean(cv_accuracy)

0.7818181818181819

# 하이퍼파라미터 튜닝

하이퍼파라미터 : 알고리즘의 성능을 튜닝하기 위해 조절하는 변수값

In [15]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
                        train_test_split(X, y,
                                        test_size=0.2,
                                        random_state=13)

In [18]:
from sklearn.model_selection import GridSearchCV

# 총 12개위 경우의 수로 학습 반복
parameters = {'max_depth':[2,3,4,5], 'min_samples_split':[2,3,4]}
grid_clf =GridSearchCV(clf, param_grid = parameters, cv=3)

In [19]:
# 학습 시작
grid_clf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [2, 3, 4, 5],
                         'min_samples_split': [2, 3, 4]})

In [20]:
scores_df = pd.DataFrame(grid_clf.cv_results_)
scores_df[['rank_test_score', 'params','mean_test_score', 'split0_test_score',
          'split1_test_score', 'split2_test_score']]

,rank_test_score,params,mean_test_score,split0_test_score,split1_test_score,split2_test_score
0,10,"{'max_depth': 2, 'min_samples_split': 2}",0.788286,0.788530,0.781362,0.794964
1,10,"{'max_depth': 2, 'min_samples_split': 3}",0.788286,0.788530,0.781362,0.794964
2,10,"{'max_depth': 2, 'min_samples_split': 4}",0.788286,0.788530,0.781362,0.794964
3,4,"{'max_depth': 3, 'min_samples_split': 2}",0.795467,0.774194,0.806452,0.805755
4,4,"{'max_depth': 3, 'min_samples_split': 3}",0.795467,0.774194,0.806452,0.805755
5,4,"{'max_depth': 3, 'min_samples_split': 4}",0.795467,0.774194,0.806452,0.805755
6,1,"{'max_depth': 4, 'min_samples_split': 2}",0.803830,0.795699,0.810036,0.805755
7,2,"{'max_depth': 4, 'min_samples_split': 3}",0.802635,0.792115,0.810036,0.805755
8,2,"{'max_depth': 4, 'min_samples_split': 4}",0.802635,0.792115,0.810036,0.805755
9,9,"{'max_depth': 5, 'min_samples_split': 2}",0.793077,0.777778,0.795699,0.805755


In [22]:
grid_clf.best_params_

{'max_depth': 4, 'min_samples_split': 2}

In [23]:
grid_clf.best_score_

0.8038300542361809

In [24]:
clf_best = grid_clf.best_estimator_

pred = clf_best.predict(X_test)
accuracy_score(y_test, pred)

0.7655502392344498

accuracy = 전체 데이터 중 맞게 예측한 것의 비율
         = $TP + TN \over {TP + TN + FP + FN}$